In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error



pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 80)

In [9]:
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

files_id = {
    'housing_data':"1kX_jcLeBpBGvTo8FXDeU2MK-aw1a0voU", #iteration7 data
    'test_housing_data':"1CMsAWhWKWBjI6DDEHtcYmRVZRazfE9bo", #test data for housing
    'ids_com':"10gwiL49calkj-xbx-3rQEK4H2zoJcU11" #ID for the commiting of the project

}

housing_data = pd.read_csv(gd_path(files_id['housing_data']), sep=",")
test_housing_data = pd.read_csv(gd_path(files_id['test_housing_data']), sep=",")
ids_com = pd.read_csv(gd_path(files_id['ids_com']), sep=",")
df = housing_data

# Pre-Processing Pipe

## Ordinary columns

This part was done by hands. Many Bothans died to bring us this information.

In [10]:
from sklearn.model_selection import train_test_split

# X and y creation
y = df.pop("SalePrice")

# Feature Engineering
X = df

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
# building the pipeline
X_cat = X.select_dtypes(exclude="number").copy()
X_num = X.select_dtypes(include="number").copy()

#  numerical pipeline
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))


# categorical pipeline
# defining ordinal & onehot columns

ordinal_cols = X_cat.columns.get_indexer(['LandContour', 'LandSlope',
                     'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                     'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                     'HeatingQC', 'KitchenQual', 'Functional',
                     'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                     'PoolQC']) 

onehot_cols = X_cat.columns.get_indexer(['MSZoning', 'Condition1', 'Heating', 'Street', 'CentralAir', 
                                         'Foundation', 'Alley', 'LotShape', 'Utilities', 'LotConfig', 
                                         'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 
                                         'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
                                         'Electrical', 'GarageType', 'PavedDrive', 'Fence', 'MiscFeature',
                                         'SaleType', 'SaleCondition'])



# defining the categorical encoder
# we manually establish the order of the categories for our ordinal feature (Cabin), including "N_A"
LandContour = ['Lvl', 'Bnk', 'HLS', 'Low']
LandSlope = ['Gtl', 'Mod', 'Sev']
ExterQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
ExterCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
BsmtQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtExposure = ['Gd', 'Av', 'Mn', 'No', 'N_A']
BsmtFinType1 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
BsmtFinType2 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
HeatingQC = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
KitchenQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
Functional = ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal']
FireplaceQu = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageFinish = ['Fin', 'RFn', 'Unf', 'N_A']
GarageQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
PoolQC = ['Ex', 'Gd', 'TA', 'Fa', 'N_A']


categorical_encoder = ColumnTransformer(
    transformers=[
        ("cat_ordinal", OrdinalEncoder(categories=[LandContour,LandSlope,
                                                    ExterQual,ExterCond, BsmtQual, BsmtCond,
                                                    BsmtExposure, BsmtFinType1, BsmtFinType2,
                                                    HeatingQC, KitchenQual, Functional,
                                                    FireplaceQu, GarageFinish, GarageQual, GarageCond,
                                                    PoolQC]), ordinal_cols),

        ("cat_onehot", OneHotEncoder(handle_unknown="ignore"), onehot_cols)
    ]
)


categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    categorical_encoder 
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num.columns),
        ("cat_pipe", categoric_pipe, X_cat.columns),
    ]
)

# Models

## Decisiontree

In [12]:
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              DecisionTreeClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14, 2),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2)
}


search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

scores = {"dtree" : search.best_score_}

scores

Fitting 5 folds for each of 60 candidates, totalling 300 fits


/Users/Smirnov/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'dtree': 0.017119694802098233}

## Decisiontree - Analysis

In [13]:
# Get predictions for the training and testing datasets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")


dtree_r2 = r2_score(y_true = y_test,
                    y_pred = y_test_pred)

print("R-squared:", round(dtree_r2,3))

dtree_r2 = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(dtree_r2,3))

Training Accuracy: 0.149
Testing Accuracy: 0.003 

R-squared: 0.359
MAPE: 0.193


## KNN


In [14]:
# Modeling Pipe - 2

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


full_pipeline = make_pipeline(preprocessor,
                              KNeighborsClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "kneighborsclassifier__n_neighbors": range(3, 20, 2)
}

search2 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=10,
                      verbose=1)

search2.fit(X_train, y_train)

scores2 = {"knn" : search.best_score_}

scores2


Fitting 10 folds for each of 18 candidates, totalling 180 fits


/Users/Smirnov/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


{'knn': 0.017119694802098233}

## KNN - Analysis

In [15]:
# Get predictions for the training and testing datasets
y_train_pred = search2.predict(X_train)
y_test_pred = search2.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")


dtree_r2 = r2_score(y_true = y_test,
                    y_pred = y_test_pred)

print("R-squared:", round(dtree_r2,3))

dtree_r2 = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(dtree_r2,3))

Training Accuracy: 0.191
Testing Accuracy: 0.007 

R-squared: 0.056
MAPE: 0.253


## RandomForest

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              RandomForestClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["median"],
    "randomforestclassifier__n_estimators": [200, 500],
    "randomforestclassifier__max_features": ['sqrt', 'log2'],
    'randomforestclassifier__max_depth' : [4,5,6,7,8],
    'randomforestclassifier__criterion' :['gini', 'entropy']
}


'''param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy": ["median"],
    "randomforestclassifier__n_estimators": [150],
    "randomforestclassifier__max_depth": [8],
    "randomforestclassifier__max_features": ["sqrt"],
    'criterion' :['gini', 'entropy']
}
'''
# need to be rearange 
search3 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search3.fit(X_train, y_train)

scores3 = {"rf" : search3.best_score_}
print("Best parameters: ", search3.best_params_)
scores3

Fitting 5 folds for each of 40 candidates, totalling 200 fits


/Users/Smirnov/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'median', 'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 4, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__n_estimators': 500}


{'rf': 0.027398114522578045}

## RandomForest - Analysis

In [17]:
# Get predictions for the training and testing datasets
y_train_pred = search3.predict(X_train)
y_test_pred = search3.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")


dtree_r2 = r2_score(y_true = y_test,
                    y_pred = y_test_pred)

print("R-squared:", round(dtree_r2,3))

dtree_r2 = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(dtree_r2,3))

Training Accuracy: 0.457
Testing Accuracy: 0.027 

R-squared: 0.675
MAPE: 0.137


### SGDRegressor

In [18]:
from sklearn.linear_model import LinearRegression, SGDRegressor

sgd_pipeline = make_pipeline(preprocessor,
                            SGDRegressor())

sgd_pipeline.fit(X_train, y_train)

sgd_predictions = sgd_pipeline.predict(X_test)

SGD_mape = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(SGD_mape,3))

SGR_r2 = r2_score(y_true = y_test,
                    y_pred = sgd_predictions)

print("R-squared:", round(SGR_r2, 3))

MAPE: 0.137
R-squared: -5.536806190171737e+24


### LinearRegression

In [19]:
# Scale the data! 

lr_pipeline = make_pipeline(preprocessor,
                            LinearRegression())

lr_pipeline.fit(X_train, y_train)

lr_predictions = lr_pipeline.predict(X_test)

lr_mape = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(lr_mape,3))

lr_r2 = r2_score(y_true = y_test,
                    y_pred = lr_predictions)

print("R-squared:", round(lr_r2,3))

MAPE: 0.137
R-squared: 0.884


# Iteration 0

In [31]:
y_test_pred_tree = search.predict(X_test)
y_test_pred_knn = search2.predict(X_test)
y_test_pred_rf = search3.predict(X_test)
y_test_pred_SGDR = sgd_pipeline.predict(X_test)
y_test_pred_lr = lr_pipeline.predict(X_test)


baseline_tree_r2 = r2_score(y_test, y_test_pred_tree)
baseline_knn_r2 = r2_score(y_test, y_test_pred_knn)
baseline_rf = r2_score(y_test, y_test_pred_rf)
baseline_SGDR = r2_score(y_test, y_test_pred_SGDR)
baseline_lr = r2_score(y_test, y_test_pred_lr)


performances = pd.DataFrame({'decision_tree': round(baseline_tree_r2,3),
                             'knn': round(baseline_knn_r2,3), 
                             'RF': round(baseline_rf,3),
                             'SGDR': round(baseline_SGDR, 3),
                            'LR': round(baseline_lr,3)},
                            index=['baseline'])

performances

,decision_tree,knn,RF,SGDR,LR
baseline,0.359,0.056,0.675,-5.536806e+24,0.884


## Variance Threshold

In [51]:
range_var_df = (pd.DataFrame({
                'Range': X_num.max() - X_num.min(),
                'Variance': round(X_num.var(),2)})
                
                .sort_values(by='Variance'))

range_var_df[range_var_df['Variance'] >=0 ]

,Range,Variance
KitchenAbvGr,3.0,0.05
BsmtHalfBath,2.0,0.06
HalfBath,2.0,0.25
BsmtFullBath,3.0,0.27
FullBath,3.0,0.30
Fireplaces,3.0,0.42
GarageCars,4.0,0.56
BedroomAbvGr,8.0,0.67
OverallCond,8.0,1.24
YrSold,4.0,1.76


# Downloand

In [ ]:
X_sumbmition = housing_data[0:1459]
ids_com['Expensive'] = search4.predict(X_sumbmition) # only cange the piplene! 
ids_com.to_csv(r'submission_9.csv', index=False) #only cahnge ist 

NameError: name 'search4' is not defined